## Preliminaries

<br>

**Packages**

In [ ]:
import subprocess

In [ ]:
if 'google.colab' in str(get_ipython()):
    subprocess.run('wget -q https://raw.githubusercontent.com/briefings/phe/develop/scripts.sh', 
                   shell=True)
    subprocess.run('chmod u+x scripts.sh', shell=True)
    subprocess.run('./scripts.sh', shell=True)

<br>

**Paths**

In [ ]:
import os
import pathlib
import sys

In [ ]:
if not 'google.colab' in str(get_ipython()):
    
    notebooks = os.getcwd()
    parent = str(pathlib.Path(notebooks).parent)
    sys.path.append(parent)
    

<br>

**Libraries**

In [ ]:
%matplotlib inline

import pandas as pd
import numpy as np

import dask
import datetime

import matplotlib.pyplot as plt
import seaborn as sns

import loggingon
import os
import glob
import requests
import collections

<br>

**Layout**

In [ ]:
sns.set_style("darkgrid")
sns.set_context("poster")
sns.set(font_scale=1)

<br>

**Logging**

In [ ]:
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

<br>
<br>

## Data

URL

* 'https://www.england.nhs.uk/statistics/wp-content/uploads/sites/2/2020/12/Covid-Publication-10-12-2020.xlsx'
* 'https://www.england.nhs.uk/statistics/wp-content/uploads/sites/2/2021/01/Covid-Publication-14-01-2021.xlsx'


In [ ]:
url = 'https://www.england.nhs.uk/statistics/wp-content/uploads/sites/2/2021/01/Covid-Publication-14-01-2021.xlsx'

<br>
<br>

### Cases

In [ ]:
import phe.cases.admissionstotal

In [ ]:
sample, institutions, notes = phe.cases.admissionstotal.AdmissionsTotal(url=url).exc()

In [ ]:
logger.info(sample.info())

In [ ]:
logger.info(institutions.info())

In [ ]:
logger.info(notes)

<br>
<br>

## Architecture

### Functions

**Epoch**

In [ ]:
def epoch(series: pd.Series):
    """

    Either
    (series - pd.Timestamp('1970-01-01')) // pd.Timedelta('1ms') 
      -> floor divide by unit
    (series.astype(np.int64) / (10 ** 6)).astype(np.longlong) 
      -> explicitly converting nanoseconds to milliseconds
    
    :param series: The series that will beconverted to epoch time (milliseconds)
    """   

    return (series.astype(np.int64) / (10 ** 6)).astype(np.longlong)
    

<br>

### Structuring

In [ ]:
Attributes = collections.namedtuple(typename='Attributes', field_names=['names'])
attributes = Attributes._make([['date', 'epoch']])

<br>

**Wide**

Via a pivot function, each institution's data series is assigned to a distinct column, i.e., a **reverse wide format**.  Subsequently, **datetime formatting & epoch calculations**.

In [ ]:
wide = sample.pivot_table(columns=['code'], 
                         values=sample.columns.drop(labels=['code']))

wide.reset_index(drop=False, inplace=True)
wide.rename(columns={'index': 'date'}, inplace=True)

In [ ]:
wide.loc[:, 'date'] = pd.to_datetime(wide['date'])
wide.loc[:, 'epoch'] = epoch(wide['date'])

<br>

**Narrow**

In [ ]:
narrow = wide.drop(columns=['epoch']).melt(id_vars=['date'], 
                                                 var_name='code', value_name='admissions')


<br>

**Inspect**

In this context the dimension attributes are `date` & `epoch`, therefore these are the fields that will be excluded from calculation procedures.

In [ ]:
expected = wide.shape[0] * (wide.shape[1] - len(attributes.names))
assert narrow.shape[0] == expected

logger.info('The expected number of value cells: {:,}'.format(expected))

<br>
<br>

## Graphing

In [ ]:
logger.info(  '\n{}\n\n'.format(wide.head().iloc[:, list(np.arange(5)) + [-1]])  )

In [ ]:
logger.info( '\n{}\n\n'.format(narrow.head()) )

<br>

### Case: HighCharts

In [ ]:
import phe.highcharts.graphseries

Path

In [ ]:
path_highcharts = os.path.join(os.getcwd(), 'data', 'highcharts')

if not os.path.exists(path_highcharts):
    os.makedirs(path_highcharts)

<br>

**JSON**

In [ ]:
phe.highcharts.graphseries.GraphSeries(blob=wide, institutions=institutions, 
                                       path=path_highcharts).exc()

<br>
<br>

### Case: Tableau

In [ ]:
import phe.tableau.graphseries

Path

In [ ]:
path_tableau = os.path.join(os.getcwd(), 'data', 'tableau')

if not os.path.exists(path_tableau):
    os.makedirs(path_tableau)

<br>

**JSON**

In [ ]:
phe.tableau.graphseries.GraphSeries(blob=wide, institutions=institutions, 
                                    path=path_tableau).exc()

In [ ]:
institutions.set_index(keys='code').to_json(
    path_or_buf=os.path.join(path_tableau, 'institutions.json'), orient='index')

<br>

**CSV**

In [ ]:
narrow.to_csv(path_or_buf=os.path.join(path_tableau, 'admissions.csv'),
              header=True, index=False, encoding='UTF-8')

institutions.to_csv(path_or_buf=os.path.join(path_tableau, 'institutions.csv'), 
                                             header=True, index=False, encoding='UTF-8')

<br>
<br>

## Zip